In [0]:
				df = spark.readStream.format("kafka") \
				.option("kafka.bootstrap.servers" , "54.221.143.212:9092") \
				.option("subscribe", "ticker_quotes")\
				.option("startingOffsets" , "earliest") \
				.option("failOnDataLoss" , "false")\
.load()

In [0]:
df.printSchema()


Trigger type ProcessingTime is not supported for this cluster type.
Use a different trigger type e.g. AvailableNow, Once. SQLSTATE: 0A000
You are in a very restrictive environment (Shared Access Mode on Databricks Community Edition). This mode has three major blocks:

Global Configs Blocked: You can't set global Hadoop keys.

Mounts Blocked: You can't mount buckets.

Root DBFS Blocked: You can't write to /dbfs/.

In [0]:


destination_path = "/Volumes/workspace/default/storage/bronze/ticker_data_v2"
# --- 2. THE FIX: PATH MUST INCLUDE THE VOLUME NAME ---
# Path format: /Volumes/<catalog>/<schema>/<VOLUME_NAME>/<folder>
# We added 'storage' because that is the volume we just created in SQL.
checkpoint_path = "/Volumes/workspace/default/storage/checkpoints/job_bronze_ticker_v2"

print(f"Streaming Strategy:")
print(f"Checkpoint: {checkpoint_path}")
print(f"Data (Volume): {destination_path}")

# --- 3. WRITE STREAM ---
df.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", checkpoint_path) \
    .trigger(availableNow=True) \
    .start(destination_path)